In [1]:
import os
import sys
import pip
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.signal import resample
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [2]:
data_path = 'data/'
train = pd.read_csv(data_path+'/sensor_train.csv')
test = pd.read_csv(data_path+'/sensor_test.csv')
sub = pd.read_csv(data_path + '/提交结果示例.csv')
y = train.groupby('fragment_id')['behavior_id'].min()

In [3]:
train['acc'] = (train.acc_x ** 2 + train.acc_y ** 2 + train.acc_z ** 2) ** .5
train['accg'] = (train.acc_xg ** 2 + train.acc_yg ** 2 + train.acc_zg ** 2) ** .5
test['acc'] = (test.acc_x ** 2 + test.acc_y ** 2 + test.acc_z ** 2) ** .5
test['accg'] = (test.acc_xg ** 2 + test.acc_yg ** 2 + test.acc_zg ** 2) ** .5

In [4]:
# accs = ['acc_x', 'acc_y', 'acc_z']
# accgs = ['acc_xg', 'acc_yg', 'acc_zg']
# two_accs = [['acc_x', 'acc_y'], ['acc_y', 'acc_z'], ['acc_x', 'acc_z']] # ['acc_y', 'acc_x'], ['acc_z', 'acc_y']
# two_accgs =  [['acc_xg', 'acc_yg'], ['acc_yg', 'acc_zg'], ['acc_xg', 'acc_zg']] # , ['acc_yg', 'acc_xg'], ['acc_zg', 'acc_yg']
# acc_and_accg = accs + accgs

for data in tqdm([train , test]):
    data['acc1'] = (data['acc_x'] ** 2 + data['acc_y'] ** 2) ** 0.5
    data['accg1'] = (data['acc_xg'] ** 2 + data['acc_yg'] ** 2) ** 0.5
    
    data['acc2'] = (data['acc_x'] ** 2 + data['acc_z'] ** 2) ** 0.5
    data['accg2'] = (data['acc_xg'] ** 2 + data['acc_zg'] ** 2) ** 0.5
    
#     data['acc3'] = (data['acc_y'] ** 2 + data['acc_z'] ** 2) ** 0.5
#     data['accg3'] = (data['acc_yg'] ** 2 + data['acc_zg'] ** 2) ** 0.5  # y - z系列 under 4%%


for data in [train, test]:
    data['acc_sub'] = ((data['acc_xg'] - data['acc_x']) ** 2 + (data['acc_yg'] - data['acc_y']) ** 2 + (data['acc_zg'] - data['acc_z'])**2) ** 0.5
    data['acc_sub1'] = ((data['acc_xg'] - data['acc_x']) ** 2 + (data['acc_yg'] - data['acc_y']) ** 2) ** 0.5
    data['acc_sub2'] = ((data['acc_xg'] - data['acc_x']) ** 2 + (data['acc_zg'] - data['acc_z'])**2) ** 0.5
#     data['acc_sub3'] = ((data['acc_yg'] - data['acc_y']) ** 2 + (data['acc_zg'] - data['acc_z'])**2) ** 0.5


for data in [train, test]:
    data['accxg_diff_accx'] = data['acc_xg'] - data['acc_x']
    data['accyg_diff_accy'] = data['acc_yg'] - data['acc_y']
    data['acczg_diff_accz'] = data['acc_zg'] - data['acc_z']
    
# for data in [train, test]:
#     data['accx_map_accy'] = data['acc_x'] * np.cos(data['acc_y'].apply(lambda x: np.abs(x)))
#     data['accx_map_accz'] = data['acc_x'] * np.cos(data['acc_z'].apply(lambda x: np.abs(x)))
    
#     data['accxg_map_accyg'] = data['acc_xg'] * np.cos(data['acc_yg'].apply(lambda x: np.abs(x)))
#     data['accxg_map_acczg'] = data['acc_xg'] * np.cos(data['acc_zg'].apply(lambda x: np.abs(x)))

100%|██████████| 2/2 [00:00<00:00, 39.97it/s]


In [5]:
# # 时间倒序
# train1 = pd.DataFrame()
# for id in tqdm(train['fragment_id'].unique()):
#     tmp = train[train['fragment_id'] == id]
#     tmp.sort_values(by = 'time_point', ascending = False, inplace = True)
#     train1 = pd.concat([train1, tmp], axis = 0)

# test1 = pd.DataFrame()
# for id in tqdm(test['fragment_id'].unique()):
#     tmp = test[test['fragment_id'] == id]
#     tmp.sort_values(by = 'time_point', ascending = False, inplace = True)
#     test1 = pd.concat([test1, tmp], axis = 0)

# train = train1
# test = test1

# train.head()

In [6]:
print(train.shape, test.shape, train.columns)

(425359, 25) (430182, 24) Index(['fragment_id', 'time_point', 'acc_x', 'acc_y', 'acc_z', 'acc_xg',
       'acc_yg', 'acc_zg', 'behavior_id', 'acc', 'accg', 'acc1', 'accg1',
       'acc2', 'accg2', 'acc_sub', 'acc_sub1', 'acc_sub2', 'accxg_diff_accx',
       'accyg_diff_accy', 'acczg_diff_accz', 'accx_map_accy', 'accx_map_accz',
       'accxg_map_accyg', 'accxg_map_acczg'],
      dtype='object')


In [7]:
x = np.zeros((7292, 60, train.shape[1] - 3, 1))
t = np.zeros((7500, 60, train.shape[1] - 3, 1))
for i in tqdm(range(7292)):
    tmp = train[train.fragment_id == i][:60]
    x[i, :, :, 0] = resample(tmp.drop(['fragment_id', 'time_point', 'behavior_id'],
                                      axis=1), 60, np.array(tmp.time_point))[0]
for i in tqdm(range(7500)):
    tmp = test[test.fragment_id == i][:60]
    t[i, :, :, 0] = resample(tmp.drop(['fragment_id', 'time_point'],
                                      axis=1), 60, np.array(tmp.time_point))[0]

100%|██████████| 7500/7500 [00:17<00:00, 428.33it/s]


In [8]:
kernal_size = 3
kfold = StratifiedKFold(5, shuffle=True)
def Net():
    input = Input(shape=(60, train.shape[1] - 3, 1))
    X = Conv2D(filters=64,
               kernel_size=(3, 3),
               activation='relu',
               padding='same')(input)
    X = BatchNormalization()(X)
    X = Conv2D(filters=128,
               kernel_size=(3, 3),
               activation='relu',
               padding='same')(X)
    X = BatchNormalization()(X)

    X = MaxPooling2D()(X)
    X = AveragePooling2D()(X)
    X = Dropout(0.2)(X)
    X = Conv2D(filters=256,
               kernel_size=(3, 3),
               activation='relu',
               padding='same')(X)
    X = BatchNormalization()(X)

    X = Dropout(0.3)(X)
    X = Conv2D(filters=512,
               kernel_size=(3, 3),
               activation='relu',
               padding='same')(X)
    X = BatchNormalization()(X)
    X = GlobalAveragePooling2D()(X)
    X = Dropout(0.5)(X)
    X = BatchNormalization()(Dropout(0.2)(Dense(128, activation='relu')(Flatten()(X))))
    X = Dense(19, activation='softmax')(X)
    return Model([input], X)



In [9]:
proba_x = np.zeros((7292, 19))
proba_t = np.zeros((7500, 19))
for fold, (xx, yy) in enumerate(kfold.split(x, y)):
    print("{}train {}th fold{}".format('==' * 30, fold + 1, '==' * 30))
    y_ = to_categorical(y, num_classes=19)
    model = Net()
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
#                   optimizer = Adam(),
                  metrics=['acc'])
    model.summary()
    plateau = ReduceLROnPlateau(monitor="val_acc",
                                verbose=1,
                                mode='max',
                                factor=0.6,
                                patience=11)
    early_stopping = EarlyStopping(monitor='val_acc',
                                   verbose=1,
                                   mode='max',
                                   patience=18)
    checkpoint = ModelCheckpoint(f'fold{fold}.h5',
                                 monitor='val_acc',
                                 verbose=0,
                                 mode='max',
                                 save_best_only=True)

#     csv_logger = CSVLogger('log.csv', separator=',', append=True)
    model.fit(x[xx], y_[xx],
              epochs=500,
              batch_size=128,
              verbose=2,
              shuffle=True,
              validation_data=(x[yy], y_[yy]),
              callbacks=[plateau, early_stopping, checkpoint])
    model.load_weights(f'fold{fold}.h5')
    proba_x[yy] += model.predict(x[yy], verbose=0, batch_size=128)
    proba_t += model.predict(t, verbose=0, batch_size=128) / 5.    
    

============================================================train 1th fold============================================================
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 60, 22, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 60, 22, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 60, 22, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 22, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 60, 22, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 11, 128)       0  

In [10]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
def acc_combo(y, y_pred):
    # 数值ID与行为编码的对应关系
    mapping = {0: 'A_0', 1: 'A_1', 2: 'A_2', 3: 'A_3', 
        4: 'D_4', 5: 'A_5', 6: 'B_1',7: 'B_5', 
        8: 'B_2', 9: 'B_3', 10: 'B_0', 11: 'A_6', 
        12: 'C_1', 13: 'C_3', 14: 'C_0', 15: 'B_6', 
        16: 'C_2', 17: 'C_5', 18: 'C_6'}
    # 将行为ID转为编码
    code_y, code_y_pred = mapping[y], mapping[y_pred]
    if code_y == code_y_pred: #编码完全相同得分1.0
        return 1.0
    elif code_y.split("_")[0] == code_y_pred.split("_")[0]: #编码仅字母部分相同得分1.0/7
        return 1.0/7
    elif code_y.split("_")[1] == code_y_pred.split("_")[1]: #编码仅数字部分相同得分1.0/3
        return 1.0/3
    else:
        return 0.0

oof_y = np.argmax(proba_x, axis=1)
score1 = round(accuracy_score(y, oof_y), 5)
# print('accuracy_score ', score1)

score = round(sum(acc_combo(y_true, y_pred) for y_true, y_pred in zip(y, oof_y)) / oof_y.shape[0], 5)
print('accuracy_score ', score1, 'accuracy_score ', score)

# ]accuracy_score  0.78799 accuracy_score  0.81902    up  757
# accuracy_score  0.79361 accuracy_score  0.82357     up  764  
# accuracy_score  0.80156 accuracy_score  0.83124
# accuracy_score  79923

# accuracy_score  0.80801 accuracy_score  0.83674
# accuracy_score  0.80965 accuracy_score  0.83811

# accuracy_score  0.80911 accuracy_score  0.8378

accuracy_score  0.80321 accuracy_score  0.83238


In [11]:
pd.DataFrame(proba_x, columns = ['pred_{}'.format(i) for i in range(19)]).to_csv(data_path + 'sub/proba_x_forward_{}.csv'.format(score1), index = False)
pd.DataFrame(proba_t, columns = ['pred_{}'.format(i) for i in range(19)]).to_csv(data_path + 'sub/proba_t_forward_{}.csv'.format(score1), index = False)

In [12]:
# pd.DataFrame(proba_x, columns = ['pred_{}'.format(i) for i in range(19)]).to_csv(data_path + 'sub/proba_x_backward_{}.csv'.format(score1), index = False)
# pd.DataFrame(proba_t, columns = ['pred_{}'.format(i) for i in range(19)]).to_csv(data_path + 'sub/proba_t_backward_{}.csv'.format(score1), index = False)

In [13]:
sub.behavior_id = np.argmax(proba_t, axis=1)
sub.to_csv(data_path + 'sub/submit_{}.csv'.format(score1), index=False)

In [14]:
# # 前向和后向融合
# proba_x_f = pd.read_csv(data_path + 'sub/proba_x_forward_0.80801.csv')
# proba_t_f = pd.read_csv(data_path + 'sub/proba_t_forward_0.80801.csv')

# proba_x_b = pd.read_csv(data_path + 'sub/proba_x_backward_0.80965.csv')
# proba_t_b = pd.read_csv(data_path + 'sub/proba_t_backward_0.80965.csv')

# proba_x_f, proba_x_b, proba_t_f, proba_t_b = np.array(proba_x_f),np.array(proba_x_b),np.array(proba_t_f),np.array(proba_t_b)

# proba_x = (proba_x_f + proba_x_b) / 2
# proba_t = (proba_t_f + proba_t_b) / 2


# oof_y = np.argmax(proba_x, axis=1)
# score1 = round(accuracy_score(y, oof_y), 5)
# # print('accuracy_score ', score1)

# score = round(sum(acc_combo(y_true, y_pred) for y_true, y_pred in zip(y, oof_y)) / oof_y.shape[0], 5)
# print('accuracy_score ', score1, 'accuracy_score ', score)

# sub.behavior_id = np.argmax(proba_t, axis=1)
# sub.to_csv(data_path + 'sub/submit_{}.csv'.format(score1), index=False)


# # accuracy_score  0.80787 accuracy_score  0.83623  up 772
# # accuracy_score  0.81418 accuracy_score  0.84175
# # accuracy_score  0.82021 accuracy_score  0.84703  up 776